In [16]:
import duckdb
from huggingface_hub import HfApi

In [17]:
api = HfApi()
con = duckdb.connect()

In [6]:
files = api.list_repo_files("allenai/c4", repo_type="dataset")

In [9]:
ml = [f for f in files if f.startswith("multilingual/") and f.endswith(".json.gz") and "-validation" not in f]

In [15]:
ml = [f"hf://datasets/allenai/c4/{i}" for i in ml]

In [20]:
ml_sample = ml[:100]